# 必要なモジュールのインポート

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from collections import defaultdict
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay

# 訓練用データと評価用データの読み込み
画像を短い辺が64ピクセルになるように全体を縮小し、その後中央の64×64ピクセルのみを切り取る。

画像の特徴量と教師データを、訓練用データと評価用データに分けて、それぞれ`X`と`y`にランダムな順番で格納する。これにより、`X['train']`には訓練用の特徴量、`y['train']`には訓練用の教師データ、`X['val']`には評価用の特徴量、`y['val']`には評価用の教師データが格納される。

特徴量は、チャネル数3 x 縦64 x 横64の数字の並びを一次元にする。

In [ ]:
root_dir = Path('./')

#
data_path = root_dir / 'training'
#

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
    ]),
}

image_datasets = {
    x: datasets.ImageFolder(root=data_path/x, transform=data_transforms[x])
    for x in ['train', 'val']}

torch.manual_seed(42)
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=1, shuffle=True)
    for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

X = defaultdict(list)
y = defaultdict(list)
for dataset in ['train', 'val']:
    for img, labels in dataloaders[dataset]:
        X[dataset].append(np.array(img).flatten())
        y[dataset].append(labels.item())
    X[dataset] = np.array(X[dataset])
    y[dataset] = np.array(y[dataset])

print('dataset size:', dataset_sizes)
print('class name:', class_names)

# 学習モデルの作成と正確度の計算を行う`learn()`関数を定義
引数で指定された`n`を`n_estimators`の値として学習モデルを作成し、訓練用データで学習を行う。

訓練用データと評価用データそれぞれについて、正確度を計算し、学習済みのモデルと正確度を返す。

In [ ]:
def learn(X, y, n):
    model = RandomForestClassifier(n_estimators=n, random_state=42, class_weight='balanced', n_jobs=-1)
    model.fit(X['train'], y['train'])

    score = {}
    for dataset in ['train', 'val']:
        score[dataset] = model.score(X[dataset], y[dataset])
    return score, model

# 評価用データで最も正確度が高くなる学習モデルを選択
`n_estimators`の値を50, 100, 500, 1000, 1500, 2000, 2500, 3000と変化させて学習モデルを作成したときに、最も評価用データの正確度が高くなり、かつ、`n_estimators`の値が小さいものを、最も良い学習モデルとして残す。

In [ ]:
best_model = None
best_val_score = 0.0
for i in (50, 100, 500, 1000, 1500, 2000, 2500, 3000):
    score, model = learn(X, y, i)
    print(f"本数：{i} 訓練用データでの正確度：{score['train']:.3f} 評価用データでの正確度：{score['val']:.3f}")
    if score['val'] > best_val_score:
        best_model = model
        best_val_score = score['val']
print()
print('best model:', best_model)
print('best score:', best_val_score)

# 混同行列による評価
最も良い学習モデルにより、評価用データの判別を行い、その結果を混同行列で評価する。

In [ ]:
predicted = best_model.predict(X['val'])
ConfusionMatrixDisplay.from_predictions(y['val'], predicted)
plt.show()

# 判別結果の可視化を行う`check_results()`関数を定義
引数として、判別に成功（`success`）、もしくは、判別に失敗（`failed`）のどちらを表示するかを示す`flag`を受け取り、それに応じて、最大40個までの該当する評価用データの画像を表示する。

In [ ]:
def check_results(Xs, ys, preds, flag):
    H = 4
    W = 10
    fig = plt.figure(figsize=(W, H))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1.0, hspace=0.4, wspace=0.4)

    i = 0
    for img, label, pred in zip(Xs, ys, preds):
        if (flag == 'success' and label == pred) or (flag == 'failed' and label != pred):
            plt.subplot(H, W, i+1)
            img = img.reshape(3, 64, 64).transpose(1, 2, 0)
            plt.imshow(img)
            plt.title(f'{label}->{pred}', fontsize=8)
            plt.axis('off')
            i += 1
            if i >= H * W:
                break
    plt.show()

# 判別に成功した画像を表示する

In [ ]:
check_results(X['val'], y['val'], predicted, 'success')

# 判別に失敗した画像を表示する

In [ ]:
check_results(X['val'], y['val'], predicted, 'failed')